In [1]:
import json
import pandas as pd

def sif_to_cytoscape_json(file_path):
    nodes = set()
    edges = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for index, line in enumerate(lines, start=1):
        parts = line.strip().split()
        if len(parts) < 3:
            raise ValueError(f"⚠️ Line {index}: Expected 'source interaction target', found '{line.strip()}'.")

        source, interaction, target = parts
        if source == target:
            raise ValueError(f"⚠️ Line {index}: Self-loop detected ('{source}' interacts with itself').")

        nodes.update([source, target])
        edge_id = f"{source}-{interaction}-{target}"

        edges.append({
            "data": {
                "id": edge_id,
                "source": source,
                "target": target,
                "interaction": interaction
            }
        })

    return {
        "elements": {
            "nodes": [{"data": {"id": node}} for node in nodes],
            "edges": edges
        }
    }

def json_to_cytoscape_json(file_path):
    with open(file_path, 'r') as file:
        input_json = json.load(file)

    if "elements" in input_json and isinstance(input_json["elements"], dict):
        return input_json

    nodes, edges = [], []

    if isinstance(input_json, list):
        for item in input_json:
            if "source" in item and "target" in item:
                edges.append({"data": {"id": f"{item['source']}-{item['target']}", **item}})
            elif "id" in item:
                nodes.append({"data": {"id": item["id"], **item}})
    elif "nodes" in input_json and "edges" in input_json:
        nodes = [{"data": {"id": node["id"], **node}} for node in input_json["nodes"]]
        edges = [{"data": {"id": edge.get("id", f"{edge['source']}-{edge['target']}"), **edge}}
                 for edge in input_json["edges"]]

    return {"elements": {"nodes": nodes, "edges": edges}}

def excel_to_cytoscape_json(file_path):
    xls = pd.ExcelFile(file_path)
    if "Nodes" not in xls.sheet_names or "Edges" not in xls.sheet_names:
        raise ValueError("Excel must contain 'Nodes' and 'Edges' sheets.")

    nodes_df = pd.read_excel(xls, sheet_name="Nodes")
    edges_df = pd.read_excel(xls, sheet_name="Edges")

    nodes = [{"data": row.dropna().to_dict()} for _, row in nodes_df.iterrows()]
    edges = [{"data": row.dropna().to_dict()} for _, row in edges_df.iterrows()]

    return {"elements": {"nodes": nodes, "edges": edges}}


In [3]:
json_to_cytoscape_json('biology_data.json')

{'elements': {'nodes': [], 'edges': []}}

In [5]:
sif_to_cytoscape_json('sample_sif.sif')

{'elements': {'nodes': [{'data': {'id': 'O'}},
   {'data': {'id': 'Y'}},
   {'data': {'id': 'T'}},
   {'data': {'id': 'H'}},
   {'data': {'id': 'X'}},
   {'data': {'id': 'E'}},
   {'data': {'id': 'P'}},
   {'data': {'id': 'G'}},
   {'data': {'id': 'M'}},
   {'data': {'id': 'C'}},
   {'data': {'id': 'Z'}},
   {'data': {'id': 'F'}},
   {'data': {'id': 'I'}},
   {'data': {'id': 'D'}},
   {'data': {'id': 'S'}},
   {'data': {'id': 'J'}},
   {'data': {'id': 'U'}},
   {'data': {'id': 'W'}},
   {'data': {'id': 'N'}},
   {'data': {'id': 'R'}},
   {'data': {'id': 'V'}},
   {'data': {'id': 'B'}},
   {'data': {'id': 'Q'}},
   {'data': {'id': 'L'}},
   {'data': {'id': 'A'}},
   {'data': {'id': 'K'}}],
  'edges': [{'data': {'id': 'A-activates-B',
     'source': 'A',
     'target': 'B',
     'interaction': 'activates'}},
   {'data': {'id': 'B-inhibits-C',
     'source': 'B',
     'target': 'C',
     'interaction': 'inhibits'}},
   {'data': {'id': 'C-interacts-D',
     'source': 'C',
     'target': 'D

In [7]:
def convert_to_cytoscape_json(input_json):
    cytoscape_elements = {"nodes": [], "edges": []}

    for element in input_json.get("elements", []):
        data = element.get("data", {})
        
        if element.get("group") == "nodes":
            node = {
                "data": {
                    "id": data.get("id"),
                    "altered": data.get("altered"),
                    "rank": data.get("rank"),
                    "cited": data.get("cited"),
                    "uniprotdesc": data.get("uniprotdesc"),
                    "isseed": data.get("isseed"),
                    "uniprot": data.get("uniprot"),
                    "isvalid": data.get("isvalid"),
                    "importance": data.get("importance")
                },
                "position": element.get("position", {"x": 0, "y": 0})
            }
            cytoscape_elements["nodes"].append(node)

        elif element.get("group") == "edges":
            edge = {
                "data": {
                    "id": data.get("id", f"{data.get('source')}_{data.get('target')}"),
                    "source": data.get("source"),
                    "target": data.get("target"),
                    "altered": data.get("altered"),
                    "rank": data.get("rank"),
                    "cited": data.get("cited"),
                    "uniprotdesc": data.get("uniprotdesc"),
                    "isseed": data.get("isseed"),
                    "uniprot": data.get("uniprot"),
                    "isvalid": data.get("isvalid"),
                    "importance": data.get("importance")
                }
            }
            cytoscape_elements["edges"].append(edge)

    return {"elements": cytoscape_elements}

In [9]:
convert_to_cytoscape_json('biology_data.json')

AttributeError: 'str' object has no attribute 'get'

In [11]:
import json

def convert_to_cytoscape_json(input_json):
    cytoscape_elements = {"nodes": [], "edges": []}

    # Iterate through elements (nodes or edges)
    for element in input_json:
        data = element.get("data", {})
        group = element.get("group")

        if group == "nodes":
            node = {
                "data": {
                    "id": data.get("id"),
                    "altered": data.get("altered"),
                    "rank": data.get("rank"),
                    "cited": data.get("cited"),
                    "uniprotdesc": data.get("uniprotdesc"),
                    "isseed": data.get("isseed"),
                    "uniprot": data.get("uniprot"),
                    "isvalid": data.get("isvalid"),
                    "importance": data.get("importance")
                },
                "position": element.get("position", {"x": 0, "y": 0})
            }
            cytoscape_elements["nodes"].append(node)

        elif group == "edges":
            edge = {
                "data": {
                    "id": data.get("id", f"{data.get('source')}_{data.get('target')}"),
                    "source": data.get("source"),
                    "target": data.get("target"),
                    "altered": data.get("altered"),
                    "rank": data.get("rank"),
                    "cited": data.get("cited"),
                    "uniprotdesc": data.get("uniprotdesc"),
                    "isseed": data.get("isseed"),
                    "uniprot": data.get("uniprot"),
                    "isvalid": data.get("isvalid"),
                    "importance": data.get("importance")
                }
            }
            cytoscape_elements["edges"].append(edge)

    return {"elements": cytoscape_elements}

# Example usage:
with open('biology_data.json', 'r') as file:
    input_json = json.load(file)

converted_json = convert_to_cytoscape_json(input_json)


In [13]:
converted_json

{'elements': {'nodes': [{'data': {'id': 'HIRA',
     'altered': 0,
     'rank': 148,
     'cited': 53,
     'uniprotdesc': 'Cooperates with ASF1A to promote replication-independentchromatin assembly. Required for the periodic repression ofhistone gene transcription during the cell cycle. Required for theformation of senescence-associated heterochromatin foci (SAHF) andefficient senescence-associated cell cycle exit.',
     'isseed': False,
     'uniprot': 'P54198',
     'isvalid': True,
     'importance': 3},
    'position': {'x': 539.5, 'y': 641.7372431003224}},
   {'data': {'id': 'SERPINE1',
     'altered': 0,
     'rank': 5,
     'cited': 7063,
     'uniprotdesc': "Serine protease inhibitor. This inhibitor acts as 'bait'for tissue plasminogen activator, urokinase, protein C andmatriptase-3/TMPRSS7. Its rapid interaction with PLAT may functionas a major control point in the regulation of fibrinolysis.",
     'isseed': False,
     'uniprot': 'P05121',
     'isvalid': True,
     'impor

In [23]:
import json

def convert_to_cytoscape_json(input_json):
    cytoscape_elements = {"nodes": [], "edges": []}

    if not isinstance(input_json, list):
        raise ValueError("Input JSON must be a list of elements.")

    for idx, element in enumerate(input_json):
        try:
            group = element.get("group")
            data = element.get("data", {})

            if not group:
                print(f"Warning: Element at index {idx} missing 'group'. Skipping.")
                continue

            if group == "nodes":
                node_id = data.get("id")
                if not node_id:
                    print(f"Warning: Node at index {idx} missing 'id'. Skipping.")
                    continue

                node = {
                    "data": {
                        "id": node_id,
                        "altered": data.get("altered"),
                        "rank": data.get("rank"),
                        "cited": data.get("cited"),
                        "uniprotdesc": data.get("uniprotdesc"),
                        "isseed": data.get("isseed"),
                        "uniprot": data.get("uniprot"),
                        "isvalid": data.get("isvalid"),
                        "importance": data.get("importance")
                    },
                    "position": element.get("position", {"x": 0, "y": 0})
                }
                cytoscape_elements["nodes"].append(node)

            elif group == "edges":
                source = data.get("source")
                target = data.get("target")
                if not source or not target:
                    print(f"Warning: Edge at index {idx} missing 'source' or 'target'. Skipping.")
                    continue

                edge = {
                    "data": {
                        "id": data.get("id", f"{source}_{target}"),
                        "source": source,
                        "target": target,
                        "altered": data.get("altered"),
                        "rank": data.get("rank"),
                        "cited": data.get("cited"),
                        "uniprotdesc": data.get("uniprotdesc"),
                        "isseed": data.get("isseed"),
                        "uniprot": data.get("uniprot"),
                        "isvalid": data.get("isvalid"),
                        "importance": data.get("importance")
                    }
                }
                cytoscape_elements["edges"].append(edge)
            else:
                print(f"Warning: Unknown group '{group}' at index {idx}. Skipping.")

        except Exception as e:
            print(f"Error processing element at index {idx}: {e}")

    return {"elements": cytoscape_elements}


def main(input_file, output_file):
    try:
        with open(input_file, 'r') as file:
            input_json = json.load(file)
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
        return
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in '{input_file}'. {e}")
        return
    except Exception as e:
        print(f"Error reading '{input_file}': {e}")
        return

    try:
        converted_json = convert_to_cytoscape_json(input_json)
    except ValueError as ve:
        print(f"Conversion error: {ve}")
        return

    try:
        with open(output_file, 'w') as outfile:
            json.dump(converted_json, outfile, indent=4)
        print(f"Conversion successful! Output saved to '{output_file}'.")
    except Exception as e:
        print(f"Error writing to '{output_file}': {e}")


# Example usage
if __name__ == "__main__":
    main('biology_data.json', 'cytoscape_data.json')


Conversion successful! Output saved to 'cytoscape_data.json'.
